In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from math import fabs
from random import randint

In [2]:
# Definindo algumas constantes
saidas = ['DOMPRECDOMEXP', 'DOMPRECRESEXP', 'ADENSEXCDOMEXP', 'ADENSEXCRESEXP',	'ONUSEXCDOMEXP', 'ONUSEXCRESEXP', 'COABFAMDOMEXP', 'COABFAMRESEXP']
saidasDomicilios = ['DOMPRECDOMEXP', 'ADENSEXCDOMEXP', 'ONUSEXCDOMEXP', 'COABFAMDOMEXP']
saidasDesidentes = ['DOMPRECRESEXP', 'ADENSEXCRESEXP', 'ONUSEXCRESEXP', 'COABFAMRESEXP']
domiciliosPrecarios = ['DOMPRECDOMEXP']
adensamentoExcessivo = ['ADENSEXCDOMEXP']
onusExcessivo = ['ONUSEXCDOMEXP']
coabtaçãoFamiliar = ['COABFAMDOMEXP']

In [ ]:
# Importa os dados
deficit_data_y = pd.read_excel('Banco_903_Amostras_Completo.xlsx')[saidasDomicilios]
deficit_data_x = pd.read_excel('Dimensoes_PCA_903_Amostras.xlsx')

# Concatena os dois blocos horizontalmente
deficit_data = pd.concat([deficit_data_x, deficit_data_y], axis=1)

# Exibe o banco de dados
deficit_data

In [4]:
# Embaralha a ordem das amostras e divide em blocos de treino (80%) e validação (20%)
train, validation = train_test_split(deficit_data, test_size=0.2, random_state=randint(0, 100))

train_x = train.drop(columns=saidasDomicilios)
validation_x = validation.drop(columns=saidasDomicilios)
train_y = train[saidasDomicilios]
validation_y = validation[saidasDomicilios]

numLines = int(validation_x.shape[0] * 0.25) #Equivalente a aproximadamente 5% do banco de dados

# Separa uma parte do bloco de validação para teste
validation_x, test_x = validation_x[:numLines], validation_x[numLines:]
validation_y, test_y = validation_y[:numLines], validation_y[numLines:]

In [ ]:
# Inicializa a rede neural
neuralNetwork = Sequential()
neuralNetwork.add(Dense(units = 24, activation = 'relu', input_dim = validation_x.shape[1]))
neuralNetwork.add(Dense(units = train_y.shape[1], activation = 'linear'))
neuralNetwork.compile(loss = 'huber', optimizer = 'rmsprop', metrics = ['mae'])

In [ ]:
# Inicia o treinamento da rede
redes_treinadas = neuralNetwork.fit(train_x, train_y, epochs = 400, batch_size = 32, validation_data = (validation_x, validation_y))

In [ ]:
# Imprime o menor erro absoluto médio encontrado
print(min(neuralNetwork.history.history['val_mae']))

# Plota o gráfico de convegência do treinamento
plt.plot(neuralNetwork.history.history['val_mae'])
plt.title('Histórico de Treinamento')
plt.ylabel('Erro Absoluto')
plt.xlabel('Épocas de treinamento')
plt.legend(['Rede Componentes Principais'])
plt.show()

In [ ]:
# Insere os valores de teste na rede e coleta os resultados gerados pela rede
predicts = neuralNetwork.predict(test_x)

In [ ]:
errorPercentage = []

# Calculo do tamanho do erro
for i in range(test_y.shape[0]):
    errorPercentage.append([])

    for j in range(test_y.shape[1]):
        erro = fabs(predicts[i][j] - test_y.iloc[i,j]) # Calcula o módulo da diferença entre o valor real e o gerado pela rede

        if test_y.iloc[i,j] != 0: # Verifica se o valor real é diferente de zero para evitar divisão por zero
            errorPercentage[i].append((erro / test_y.iloc[i,j])*100) # Calcula a porcentagem do tamanho do erro em relaçao ao valor real

        else:
            errorPercentage[i].append(None)  # Substitui a porcentagem com um valor nulo para ser descartado

# Exibe o resultado
for i in errorPercentage:
    print(i)

In [ ]:
meanErrorPercentage = []

# Calcula a média das porcentagens excluindo os valores nulos
for i in range(test_y.shape[1]):
    sum = 0
    counter = test_y.shape[0]

    for j in range(test_y.shape[0]):
        if errorPercentage[j][i] == None:
            counter -= 1

        else:
            sum += errorPercentage[j][i]

    meanErrorPercentage.append(sum/counter)

print("Média do Erro em Porcetagem")
for i in range(len(meanErrorPercentage)):
    print("{}: {:.2f}%".format(saidasDomicilios[i], meanErrorPercentage[i]))